## Setup

In [15]:
# analytics ----
from siuba.sql import LazyTbl
from siuba import *
from sqlalchemy import create_engine
import pandas as pd

# widgets ----
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, FileLink

# warnings ----
import warnings

warnings.filterwarnings("ignore")

In [2]:
engine = create_engine("bigquery://cal-itp-data-infra")

## Create tables

In [3]:
tbl_status = LazyTbl(engine, "gtfs_schedule.calitp_status")
tbl_validation_notices = LazyTbl(engine, 'test_views.validation_notices')
sheet = pd.read_csv("https://docs.google.com/spreadsheets/d/105oar4q_Z3yihDeUlP-VnYpJ0N9Mfs7-q4TnribqYLU/gviz/tq?tqx=out:csv&gid=471807468")

/opt/conda/lib/python3.9/site-packages/google/cloud/bigquery/client.py:461: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:524: SAWarning: Did not recognize type 'BIGNUMERIC' of column 'shapeDistTraveled'
  util.warn(
/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:524: SAWarning: Did not recognize type 'BIGNUMERIC' of column 'prevShapeDistTraveled'
  util.warn(
/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:524: SAWarning: Did not recognize type 'BIGNUMERIC' of column 'prevStopTimeDistTraveled'
  util.warn(
/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:524: SAWarning: Did not recognize type 'BIGNUMERIC' of column 'speedkmh'
  util.warn(


## Fetch district specific validator results

In [4]:
def get_district_validations(district, agency_itp_id=None):
    if agency_itp_id is None:
        df_district = sheet[lambda d: d["Caltrans District"] == district]
        itp_ids = df_district.ITP_ID.astype(int).tolist()
    else:
        itp_ids = [agency_itp_id]

    itp_id_str = ", ".join(map(str, itp_ids))

    tbl_notices_subset = (
        tbl_validation_notices
        >> filter(_.calitp_itp_id.isin(itp_ids))
        >> left_join(
            _,
            tbl_status >> select(_.itp_id, _.url_number, _.agency_name),
            {"calitp_itp_id": "itp_id", "calitp_url_number": "url_number"},
        )
        # move useful columns to the left
        >> select(_.calitp_itp_id, _.calitp_url_number, _.agency_name, _.contains(""))
    )
    
    return tbl_notices_subset >> collect()


In [12]:
def ui_download_validations(district, agency, output):
    with output:
        if district is None:
            print("Select a district to download")
            return

        display("getting results...")
        df = get_district_validations(district)

        fname = "./district_%s_validations.csv" % int(district)
        df.to_csv(fname)
        
        display(FileLink(fname, result_html_prefix="Download file: "))

In [13]:
options = sheet["Caltrans District"].dropna().sort_values().astype(int).unique()

w_district = widgets.Dropdown(options = [None, *options])
w_agency = widgets.Dropdown()
w_run = widgets.Button(description="Run")
w_out = widgets.Output()


def update_agency(change):
    df_district = sheet[sheet["Caltrans District"] == w_district.value]
    agency_ids = df_district.sort_values("Agency")[["Agency", "ITP_ID"]].dropna()
    
    itp_ids = agency_ids["ITP_ID"].astype(int)
    agencies = agency_ids["Agency"]
    
    w_agency.options = [("All agencies", None), *zip(agencies, itp_ids)]
    w_agency.index = 0

w_district.observe(update_agency, "value")

w_run.on_click(
    lambda btn: ui_download_validations(w_district.value, w_agency.value, w_out)
)

In [14]:
display(w_district, w_agency, w_run, w_out)

Dropdown(options=(None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=None)

Dropdown(options=(), value=None)

Button(description='Run', style=ButtonStyle())

Output()